- Pandas的基本特性之一就是高性能的内存式数据连接（join）与合并（merge）操作。如果你有使用数据库的经验，那么对这类操作一定很熟悉。Pandas的主接口是pd.merge函数，下面让我们通过一些示例来介绍它的用法。

## 关系代数
* pd.merge()实现的功能基于关系代数（relational algebra）的一部分。关系代数是处理关系型数据的通用理论，绝大部分数据库的可用操作都以此为理论基础。关系代数方法论的强大之处在于，它提出的若干简单操作规则经过组合就可以为任意数据集构建十分复杂的操作。借助在数据库或程序里已经高效实现的基本操作规则，你可以完成许多非常复杂的操作。
* Pandas在pd.merge()函数与Series和DataFrame的join()方法里实现了这些基本操作规则。

#### 数据连接的类型
- pd.merge()函数实现了三种数据连接的类型：一对一、多对一和多对多。这三种数据连接类型都通过pd.merge()接口进行调用，根据不同的数据连接需求进行不同的操作。下面将通过一些示例来演示这三种类型，并进一步介绍更多的细节。

In [3]:
# 一对一连接
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [4]:
# 将这两个dataframe合并成一个，用pd.merge()实现
df3=pd.merge(df1,df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


pd.merge()方法会发现两个DataFrame都有“employee”列，并会自动以这列作为键进行连接。两个输入的合并结果是一个新的DataFrame。需要注意的是，共同列的位置可以是不一致的。例如在这个例子中，虽然df1与df2中“employee”列的位置是不一样的，但是pd.merge()函数会正确处理这个问题。另外还需要注意的是，pd.merge()会默认丢弃原来的行索引，不过也可以自定义

### 多对一连接
- 多对一连接是指，在需要连接的两个列中，有一列的值有重复。通过多对一连接获得的结果DataFrame将会保留重复值。

In [5]:
import pandas as pd
import numpy as np
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                           'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3)
print(df4)
print(pd.merge(df3, df4))
pd.merge(df3, df4)

#在结果DataFrame中多了一个“supervisor”列，里面有些值会因为输入数据的对应关系而有所重复

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


### 多对多连接
- 多对多连接是个有点儿复杂的概念，不过也可以理解。如果左右两个输入的共同列都包含重复值，那么合并的结果就是一种多对多连接。用一个例子来演示可能更容易理解。来看下面的例子，里面有一个DataFrame显示不同岗位人员的一种或多种能力。

In [6]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                                     'Engineering', 'Engineering', 'HR', 'HR'],

                           'skills': ['math', 'spreadsheets', 'coding', 'linux',
                                      'spreadsheets', 'organization']})
print(df1)
print(df5)
print(pd.merge(df1, df5))

# 这三种数据连接类型可以直接与其他Pandas工具组合使用，从而实现各种各样的功能。但是工作中的真实数据集往往并不像示例中演示的那么干净、整洁。


  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


## 设置数据合并的键
- 我们已经见过pd.merge()的默认行为：它会将两个输入的一个或多个共同列作为键进行合并。但由于两个输入要合并的列通常都不是同名的，因此pd.merge()提供了一些参数处理这个问题。

In [7]:
# 1. 参数on的用法
# 最简单的方法就是直接将参数on设置为一个列名字符串或者一个包含多列名称的列表
print(df1)
print(df2)
print(pd.merge(df1, df2, on='employee'))
# 
我们已经见过pd.merge()的默认行为：它会将两个输入的一个或多个共同列作为键进行合并。但由于两个输入要合并的列通常都不是同名的，因此pd.merge()提供了一些参数处理这个问题。
1. 参数on的用法
最简单的方法就是直接将参数on设置为一个列名字符串或者一个包含多列名称的列表：
In[6]: print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))

df1                          df2
  employee        group          employee hire_date
0      Bob   Accounting        0     Lisa      2004
1     Jake  Engineering        1      Bob      2008
2     Lisa  Engineering        2     Jake      2012
3      Sue           HR        3      Sue      2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
# 这个参数只能在两个DataFrame有共同列名的时候才可以使用。          

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


In [11]:
# 2.left_on与right_on参数
# 有时你也需要合并两个列名不同的数据集，例如前面的员工信息表中有一个字段不是“employee”而是“name”。在这种情况下，就可以用left_on和right_on参数来指定列名：

df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1)
print(df3);
print(pd.merge(df1, df3, left_on="employee",right_on="name"))
# 获取的结果中会有一个多余的列，可以通过DataFrame的drop()方法将这列去掉：
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000


,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


In [12]:
# 3. left_index与right_index参数
#除了合并列之外，你可能还需要合并索引。就像下面例子中的数据那样：
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a)
print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


In [13]:
# 可以通过设置pd.merge()中的left_index和/或right_index参数将索引设置为键来实现合并：
print(df1a)
print(df2a);
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


#### 为了方便考虑，DataFrame实现了join()方法，他可以按照索引进行数据合并

In [14]:
print(df1a)
print(df2a)
print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


# 设置数据连接的集合操作规则

- 通过前面的实例，我们总结出数据连接的一个重要条件：集合操作规则。当一个值出现在一列，却没有出现在另一列时，就要考虑操作规则了

In [15]:
import pandas as pd
import numpy as np
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                            'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
print(df6)
print(df7)
print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


我们合并两个数据集，在“name”列中只有一个共同的值：Mary。默认情况下，结果中只会包含两个输入集合的交集，这种连接方式被称为内连接（inner join）。我们可以用how参数设置连接方式，默认值为'inner'：

In [16]:
pd.merge(df6,df7,how='inner')

,name,food,drink
0,Mary,bread,wine


how参数支持的数据连接方式还有'outer'、'left'和'right'。外连接（outer join）返回两个输入列的交集，所有缺失值都用NaN填充：

In [17]:
# 左连接（left join）和右连接（right join）返回的结果分别只包含左列和右列
print(df6)
print(df7)
print(pd.merge(df6,df7,how='left'))
# 现在输出的行中只包含左边输入列的值。如果用how='right'的话，输出的行则只包含右边输入列的值。


    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


In [18]:
print(df6)
print(df7)
print(pd.merge(df6,df7,how='right'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


## 重复列名：suffixes参数
- 最后，你可能会遇到两个输入DataFrame有重名列的情况

In [19]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
print(df8)
print(df9)
print(pd.merge(df8, df9, on="name"))
# 由于输出结果中有两个重复的列名，因此pd.merge()函数会自动为它们增加后缀_x或_y

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [21]:
# 当然也可以通过suffixes参数自定义后缀名：
print(df8)
print(df9)
print(pd.merge(df8,df9,on="name",suffixes=[1,2]))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank1  rank2
0   Bob      1      3
1  Jake      2      1
2  Lisa      3      4
3   Sue      4      2




## 案例：美国各州的统计数据
- 数据的合并与连接是组合来源不同的数据的最常用方法。下面通过美国各州的统计数据来进行一个演示，请到https://github.com/jakevdp/data-USstates/ 下载数据：

In [3]:
import pandas as pd
import numpy as np
pop = pd.read_csv(r'./data/state-population.csv')
areas = pd.read_csv(r"./data/state-areas.csv")
abbrevs = pd.read_csv(r"./data/state-abbrevs.csv")
print("========================================")
print(pop.head())
print("========================================")
print(areas.head())
print("========================================")
print(abbrevs.head())
print("========================================")

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


### 美国各州的人口密度排名。
- 首先用一个多对一合并获取人口（pop）DataFrame中各州名称缩写对应的全称。我们需要将pop的state/region列与abbrevs的abbreviation列进行合并，还需要通过how='outer'确保数据没有丢失。

In [4]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # 丢弃重复信息
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [5]:
# 检查数据是否有缺失的
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [6]:
# 部分populatiuon是缺失值
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


    好像所有的人口缺失值都出现在2000年之前的波多黎各，此前并没有统计过波多黎各的人口。
    更重要的是，我们还发现一些新的州的数据也有缺失，可能是由于名称缩写没有匹配上全程！来看看究竟是哪个州有缺失：

In [7]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

    我们可以快速解决这个问题：人口数据中包含波多黎各（PR）和全国总数（USA），但这两项没有出现在州名称缩写表中。来快速填充对应的全称：

In [8]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [9]:
# 让我们用类似的规则将面积数据也合并进来。用两个数据集共同的state列来合并：
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [10]:
# 再检查一下数据，看看哪些列还有缺失值，没有匹配上：
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [11]:
# 面积area列里面还有缺失值。来看看究竟是哪些地区面积缺失：
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

    我们发现面积（areas）DataFrame里面不包含全美国的面积数据。可以插入全国总面积数据（对各州面积求和即可），但是针对本案例，我们要去掉这个缺失值，因为全国的人口密度在此无关紧要

In [12]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [13]:
# 先选择2000年的各州人口以及总人口数据。让我们用query()函数进行快速计算（这需要用到numexpr程序库

data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [14]:
# 现在来计算人口密度并按序排列。首先对索引进行重置，然后再计算结果：
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

    计算结果是美国各州加上华盛顿特区（Washington, DC）、波多黎各在2010年的人口密度排序，以万人 / 平方英里为单位。我们发现人口密度最高的地区是华盛顿特区的哥伦比亚地区（the District of Columbia）。在各州的人口密度中，新泽西州（New Jersey）是最高的。

In [15]:
# 还可以看看人口密度最低的几个州的数据：
density.tail()
#可以看出，人口密度最低的州是阿拉斯加（Alaska），刚刚超过1万人 / 平方英里。

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

    当人们用现实世界的数据解决问题时，合并这类脏乱的数据是十分常见的任务。希望这个案例可以帮你把前面介绍过的工具串起来，从而在数据中找到想要的答案！